<a href="https://colab.research.google.com/github/radhakrishnan-omotec/football-repo/blob/main/project_part1_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration
Before we get into building our xG model, we need to consider what sort of data we are interested in. Obviously, we need a large collection of shot data but more importantly we need the data to describe the type of shots that result in goals. We can deduce that the most important factors we need would be the distance from goal when the shot was taken, the angle with respect to the goal and what part of the body the shot was taken with.


In [1]:
!git clone https://github.com/radhakrishnan-omotec/football-repo.git

Cloning into 'football-repo'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 39 (delta 19), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (39/39), 1.39 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (19/19), done.


**Data Extraction and Wrangling**

First we have to import all the files that contain our event data. We will also import our libraries in the cell below.

In [2]:
import pandas as pd
import numpy as np
import os
import json
import requests  # Added for downloading dataset
from urllib.parse import urlparse  # Added for parsing URL
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Arc

In [4]:
# Function to download dataset from URL and save it to the specified path
def download_dataset(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

# Specify the URL and save path
dataset_url = "https://figshare.com/ndownloader/files/14464685"
save_path = "/content/football-repo/events.zip"


# Download the dataset
download_dataset(dataset_url, save_path)

# Add code to extract the contents if needed

# Specify the directory where the extracted files are stored
directory = '/content/football-repo'

# Create a list of the json files from the directory
eventjsonfiles = []
for path in os.listdir(directory):
    eventjsonfiles.append(os.path.join(directory, path))

print(eventjsonfiles)

['/content/football-repo/events.zip', '/content/football-repo/Practice_Project_Football_Goal_Prediction.ipynb', '/content/football-repo/.git', '/content/football-repo/Part1_Data_Exploration.ipynb', '/content/football-repo/README.md', '/content/football-repo/Course_work_1_Indradhar_Paka.ipynb', '/content/football-repo/Predicting the Price of a Football Player.csv', '/content/football-repo/Linear regression.ipynb', '/content/football-repo/Report_Course Work1_Indradhar Paka.pdf', '/content/football-repo/Advertising.csv']


In [5]:
import zipfile

# Specify the directory where the extracted files should be stored
extracted_directory = '/content/football-repo/extracted'

# Specify the path to the downloaded zip file
zip_file_path = '/content/football-repo/events.zip'

# Create the directory if it doesn't exist
os.makedirs(extracted_directory, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_directory)

# Create a list of the extracted json files
jsonfiles = []
for path in os.listdir(extracted_directory):
    jsonfiles.append(os.path.join(extracted_directory, path))

print("List of extracted JSON files:")
print(jsonfiles)

List of extracted JSON files:
['/content/football-repo/extracted/events_World_Cup.json', '/content/football-repo/extracted/events_England.json', '/content/football-repo/extracted/events_Spain.json', '/content/football-repo/extracted/events_Italy.json', '/content/football-repo/extracted/events_France.json', '/content/football-repo/extracted/events_European_Championship.json', '/content/football-repo/extracted/events_Germany.json']


Now we are going to parse through the json files and extract all the relevent shot data to store in a tiddy seperate dataframe.

**Most of this will be done using the pandas and numpy library.**

In [6]:
def shot_matrix(eventdata):
    with open(eventdata) as f:
        data = json.load(f)

    #lets create the dataframe that we want to store our data in and all the attributes we are interested in
    shots_dataset = pd.DataFrame(columns=['Goal','x','y','playerid','teamid','matchid','header'])

    #remember that the jsonfiles include passes, shots, tackles etc so we need to filter through these
    #lets find all the occurences of a shot within the set
    #refer to link in the prevous cell for info on the Wyscout event dataset, including tag names
    event_df = pd.DataFrame(data)
    all_shots = event_df[event_df['subEventName']=='Shot']

    #now we need to fill in our shots_dataset matrix by attribute columns
    #we will do this by filtering through the all-shot df (dataframe) we just made
    for index,shot in all_shots.iterrows():
        #here we fill in the columns for goals and headers with binary descripters
        shots_dataset.at[index,'Goal']=0
        shots_dataset.at[index,'header']=0
        for tag in shot['tags']:
            if tag['id']==101:
                shots_dataset.at[index,'Goal']=1
            elif tag['id']==403:
                shots_dataset.at[index,'header']=1

        #now we are interested in distance from the goal as well as the angle formed with the goal
        #Wyscouts pitch has its origin at the top left of the pitch and is 100m x 100m
        #therefore x and y represent percentage of nearness to top left corner
        #most pitches are 105 meters by 68 so we will go with that
        shots_dataset.at[index,'Y']=shot['positions'][0]['y']*.68
        shots_dataset.at[index,'X']= (100 - shot['positions'][0]['x'])*1.05

        #now we use dummy variables x and y to calc distance and angle attributes
        shots_dataset.at[index,'x']= 100 - shot['positions'][0]['x']
        shots_dataset.at[index,'y']=shot['positions'][0]['y']
        shots_dataset.at[index,'Center_dis']=abs(shot['positions'][0]['y']-50)

        x = shots_dataset.at[index,'x']*1.05
        y = shots_dataset.at[index,'Center_dis']*.68
        shots_dataset.at[index,'Distance'] = np.sqrt(x**2 + y**2)

        #we are interested in the angle made between the width of the goal and the
        #straight line distance to the shot location. A goal is 7.32 meters wide
        #use the law of cosines
        c=7.32
        a=np.sqrt((y-7.32/2)**2 + x**2)
        b=np.sqrt((y+7.32/2)**2 + x**2)
        k = (c**2-a**2-b**2)/(-2*a*b)
        gamma = np.arccos(k)
        if gamma<0:
            gamma = np.pi + gamma
        shots_dataset.at[index,'Angle Radians'] = gamma
        shots_dataset.at[index,'Angle Degrees'] = gamma*180/np.pi

        #lastly we add the identifiers for player, team and match
        shots_dataset.at[index,'playerid']=shot['playerId']
        shots_dataset.at[index,'matchid']=shot['matchId']
        shots_dataset.at[index,'teamid']=shot['teamId']

        print("shots_dataset created :: Counter Index : ", index)
    return shots_dataset

In [7]:
#Now we read in our json files into our shot_matrix function
all_leagues = []
for file in jsonfiles:
    all_leagues.append(shot_matrix(file))
df = pd.concat(all_leagues)

Streaming output truncated to the last 5000 lines.
shots_dataset created :: Counter Index :  26227
shots_dataset created :: Counter Index :  26287
shots_dataset created :: Counter Index :  26317
shots_dataset created :: Counter Index :  26328
shots_dataset created :: Counter Index :  26350
shots_dataset created :: Counter Index :  26418
shots_dataset created :: Counter Index :  26420
shots_dataset created :: Counter Index :  26447
shots_dataset created :: Counter Index :  26509
shots_dataset created :: Counter Index :  26545
shots_dataset created :: Counter Index :  26750
shots_dataset created :: Counter Index :  26757
shots_dataset created :: Counter Index :  26853
shots_dataset created :: Counter Index :  26912
shots_dataset created :: Counter Index :  26972
shots_dataset created :: Counter Index :  27183
shots_dataset created :: Counter Index :  27194
shots_dataset created :: Counter Index :  27272
shots_dataset created :: Counter Index :  27352
shots_dataset created :: Counter Inde

<ipython-input-6-976e7e169a77>:49: RuntimeWarning: invalid value encountered in arccos
  gamma = np.arccos(k)


Streaming output truncated to the last 5000 lines.
shots_dataset created :: Counter Index :  128657
shots_dataset created :: Counter Index :  128933
shots_dataset created :: Counter Index :  128975
shots_dataset created :: Counter Index :  129057
shots_dataset created :: Counter Index :  129142
shots_dataset created :: Counter Index :  129166
shots_dataset created :: Counter Index :  129267
shots_dataset created :: Counter Index :  129361
shots_dataset created :: Counter Index :  129528
shots_dataset created :: Counter Index :  129539
shots_dataset created :: Counter Index :  129543
shots_dataset created :: Counter Index :  129771
shots_dataset created :: Counter Index :  129853
shots_dataset created :: Counter Index :  129960
shots_dataset created :: Counter Index :  129968
shots_dataset created :: Counter Index :  129979
shots_dataset created :: Counter Index :  130019
shots_dataset created :: Counter Index :  130081
shots_dataset created :: Counter Index :  130203
shots_dataset crea

<ipython-input-6-976e7e169a77>:49: RuntimeWarning: invalid value encountered in arccos
  gamma = np.arccos(k)


Streaming output truncated to the last 5000 lines.
shots_dataset created :: Counter Index :  74257
shots_dataset created :: Counter Index :  74266
shots_dataset created :: Counter Index :  74315
shots_dataset created :: Counter Index :  74330
shots_dataset created :: Counter Index :  74336
shots_dataset created :: Counter Index :  74356
shots_dataset created :: Counter Index :  74382
shots_dataset created :: Counter Index :  74449
shots_dataset created :: Counter Index :  74679
shots_dataset created :: Counter Index :  74790
shots_dataset created :: Counter Index :  74810
shots_dataset created :: Counter Index :  74813
shots_dataset created :: Counter Index :  74846
shots_dataset created :: Counter Index :  74950
shots_dataset created :: Counter Index :  74973
shots_dataset created :: Counter Index :  75047
shots_dataset created :: Counter Index :  75091
shots_dataset created :: Counter Index :  75142
shots_dataset created :: Counter Index :  75247
shots_dataset created :: Counter Inde

<ipython-input-6-976e7e169a77>:49: RuntimeWarning: invalid value encountered in arccos
  gamma = np.arccos(k)


shots_dataset created :: Counter Index :  368095
shots_dataset created :: Counter Index :  368388
shots_dataset created :: Counter Index :  368535
shots_dataset created :: Counter Index :  368673
shots_dataset created :: Counter Index :  368714
shots_dataset created :: Counter Index :  368873
shots_dataset created :: Counter Index :  368917
shots_dataset created :: Counter Index :  368992
shots_dataset created :: Counter Index :  369157
shots_dataset created :: Counter Index :  369239
shots_dataset created :: Counter Index :  369267
shots_dataset created :: Counter Index :  369496
shots_dataset created :: Counter Index :  369530
shots_dataset created :: Counter Index :  369567
shots_dataset created :: Counter Index :  369635
shots_dataset created :: Counter Index :  369656
shots_dataset created :: Counter Index :  369920
shots_dataset created :: Counter Index :  369977
shots_dataset created :: Counter Index :  370053
shots_dataset created :: Counter Index :  370172
shots_dataset create

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43078 entries, 117 to 519397
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Goal           43078 non-null  object 
 1   x              43078 non-null  object 
 2   y              43078 non-null  object 
 3   playerid       43078 non-null  object 
 4   teamid         43078 non-null  object 
 5   matchid        43078 non-null  object 
 6   header         43078 non-null  object 
 7   Y              43078 non-null  float64
 8   X              43078 non-null  float64
 9   Center_dis     43078 non-null  float64
 10  Distance       43078 non-null  float64
 11  Angle Radians  43075 non-null  float64
 12  Angle Degrees  43075 non-null  float64
dtypes: float64(6), object(7)
memory usage: 4.6+ MB


In [9]:
df.describe()

,Y,X,Center_dis,Distance,Angle Radians,Angle Degrees
count,43078.000000,43078.000000,43078.000000,43078.000000,43075.000000,43075.000000
mean,33.473875,15.992062,11.459260,18.592949,0.414019,23.721566
std,9.366242,8.534094,7.681202,8.419041,0.252483,14.466236
min,0.000000,0.000000,0.000000,0.680000,0.000000,0.000000
25%,26.520000,9.450000,5.000000,12.249445,0.250188,14.334692
50%,33.320000,13.650000,11.000000,17.153297,0.327782,18.780499
75%,40.800000,23.100000,17.000000,24.936000,0.505984,28.990776
max,68.000000,103.950000,50.000000,103.952224,3.141593,180.000000


In [10]:
df.head()

,Goal,x,y,playerid,teamid,matchid,header,Y,X,Center_dis,Distance,Angle Radians,Angle Degrees
117,0,13,27,122940,16521,2057954,0,18.36,13.65,23.0,20.758904,0.234886,13.458001
154,0,10,69,101699,14358,2057954,0,46.92,10.50,19.0,16.648616,0.283528,16.244976
197,0,14,30,101857,14358,2057954,0,20.40,14.70,20.0,20.026233,0.270761,15.513438
232,1,7,60,102157,14358,2057954,1,40.80,7.35,10.0,10.013116,0.554534,31.772473
372,0,14,38,122671,16521,2057954,0,25.84,14.70,12.0,16.812959,0.380161,21.781597


from matplotlib import pyplot as plt
_df_0['Y'].plot(kind='hist', bins=20, title='Y')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['X'].plot(kind='hist', bins=20, title='X')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Center_dis'].plot(kind='hist', bins=20, title='Center_dis')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Distance'].plot(kind='hist', bins=20, title='Distance')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['teamid'].value_counts()
    for x_label, grp in _df_19.groupby('playerid')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('playerid')
_ = plt.ylabel('teamid')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Goal'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Y', y='Goal', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['x'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='Y', y='x', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_22['y'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_22, x='Y', y='y', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_23['playerid'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_23, x='Y', y='playerid', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)